# Biigle Exploration

This notebooks is part of the Spyfish Aoteraoa marine reserve reporting automation efforts. 
Biigle is one of the options considered as the annotation tool for experts.

It is used to explore the biigle API, especially connecting to the data in the S3 bucket and export of the annotations.
https://biigle.de/

In [ ]:
# Can Use biigle API wrapper

import os
import zipfile
import pandas as pd

from sftk.common import BIIGLE_API_EMAIL, BIIGLE_API_TOKEN


# Add biigle to your path (get from here: https://github.com/biigle/community-resources)
import sys
sys.path.append("/path/path/community-resources-master/biigle")

from biigle import Api

TODO 
- is it possible to limit the visible annotation options in the biigle frontend (for example limit to just square annotations?)
- how to mark when a video has been reviewed

In [ ]:
api = Api(BIIGLE_API_EMAIL, BIIGLE_API_TOKEN)
volumes = api.get("volumes")
volumes.json()

In [97]:
HIROMI_TEST_VOLUME_ID = 24940 # hiromi test
LOBSTER_VOLUME_ID = 20177
PROJECT_ID = 3711 # spyfish aotearoa project

# Your BIIGLE User Disk ID (get from UI or API)
# s3 bucket reference (one and can find it from existing volumes, by calling get volumes on exisiting volume or from editing sorage disk in gui)
DISK_ID = 98  

In [ ]:
volume_info = api.get(f"volumes/{HIROMI_TEST_VOLUME_ID}") # hiromi test
volume_info.json()

In [ ]:

# Get all projects that the user can access.
projects = api.get('projects').json()
projects

In [57]:
# example files 
# TODO get list automatically from AWS bucket

files = [
"TON_20211026_BUV_TON_046_01.mp4_clip_1770_10.mp4", 
"TON_20211026_BUV_TON_046_01.mp4_clip_1780_10.mp4", 
"TON_20211026_BUV_TON_046_01.mp4_clip_1790_10.mp4", 
"TON_20211026_BUV_TON_046_01.mp4_clip_1800_10.mp4", 
"TON_20211026_BUV_TON_046_01.mp4_clip_1810_10.mp4", 
"TON_20211026_BUV_TON_046_01.mp4_clip_1820_10.mp4", 
"TON_20211026_BUV_TON_046_01.mp4_clip_1830_10.mp4", 
"TON_20211026_BUV_TON_046_01.mp4_clip_1840_10.mp4", 
"TON_20211026_BUV_TON_046_01.mp4_clip_1850_10.mp4", 
"TON_20211026_BUV_TON_046_01.mp4_clip_1860_10.mp4", 
]

In [ ]:
# Create pending volume
pending_volume_response = api.post(f"projects/{PROJECT_ID}/pending-volumes", json={"media_type":"video"})
pending_volume_info = pending_volume_response.json()
pending_volume_info

In [58]:
pending_volume_id = pending_volume_info["id"]
# if the above fails: 
# find pending volume in gui by trying to make a new volume, then it tells you you can only have one pending volume, and the id is in the url

# made with the DISK_ID value
s3_url = "disk-98://biigle_test"
fill_pv = api.put(f"pending-volumes/{pending_volume_id}", json={"name": "biigle_test_1515", "url": s3_url, "files": files})
fill_pv.json()

In [ ]:
# Some extra potentially useful calls

response_volume_files = api.get(f"volumes/{HIROMI_TEST_VOLUME_ID}/files")
response_volume_files.json()

In [ ]:

response_project = api.get(f"projects/{PROJECT_ID}/")
response_project.json()

Type Ids given by biigle support: 

TODO add type ids

In [ ]:
# TODO try this
type_id = 8
project_report = api.post(f"projects/{PROJECT_ID}/reports", json={"type_id": type_id})


In [ ]:
type_id = 8
volume_annotations_report = api.post(f"volumes/{HIROMI_TEST_VOLUME_ID}/reports", json={"type_id": type_id})

volume_annotations_report.json()

In [ ]:


type_id = 10

volume_report = api.post(f"volumes/{HIROMI_TEST_VOLUME_ID}/reports", json={"type_id": type_id})

volume_report.json()

In [130]:
volume_report_id = volume_report.json()["id"]
volume_annotations_report_id = volume_annotations_report.json()["id"]
file_report =  api.get(f"reports/{volume_annotations_report_id}")

In [ ]:

zip_file_path = "biigle_annotations.zip"
with open(zip_file_path, "wb") as file:
    file.write(file_report.content)

extract_to_directory = "extracted_biigle_annotations"

# Create the extraction directory if it doesn't exist
os.makedirs(extract_to_directory, exist_ok=True)

# TODO logging
try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # TODO overwriting probably not a problem if each video will have it's dropID
        zip_ref.extractall(extract_to_directory)
    print(f"Files extracted to: {extract_to_directory}")
except zipfile.BadZipFile:
    print("Error: The downloaded file is not a valid zip file.")
except FileNotFoundError:
    print(f"Error: Zip file not found at {zip_file_path}")


In [ ]:
# TODO not sure this is the output...
annotations_df = pd.read_csv(os.path.join(extract_to_directory, f"{HIROMI_TEST_VOLUME_ID}-{ volume_annotations_report.json()["source_name"].lower()}.csv"))
annotations_df.sample()

In [ ]:
annotations_df[annotations_df["shape_name"] == "Circle"]

Extra

In [ ]:
# Direct api call example 
import requests

# your request url:
url = "https://biigle.de/api/v1/projects"
# send the request to the API
request = requests.get(url,
auth=(BIIGLE_API_EMAIL,BIIGLE_API_TOKEN))
request.json()